<h3> Segmenting and Clustering Neighborhoods in Toronto <h3>

<h4> 1. Import and Install <h4>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

<h4> 2. Scrape postal codes of Canada wiki <h4>

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

In [10]:
soup = BeautifulSoup(data, 'xml')
table = soup.find('table')

<h4> 3. Create dataframe <h4>

In [12]:
# Provide column names and create dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [15]:
# Loop through data
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [23]:
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


<h5> Remove "Not Assigned" <h5>

In [27]:
# Take a look.
df = df[df['Borough']!='Not assigned']
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [34]:
# Group and join neighborhoods
df2 = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2 = df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_Combined'},inplace=True)

In [36]:
# Merge,drop duplicates, restore neighborhoods column title
df_combined = pd.merge(df, df2, on='Postalcode')
df_combined.drop(['Neighborhood'],axis=1,inplace=True)
df_combined.drop_duplicates(inplace=True)
df_combined.rename(columns={'Neighborhood_Combined':'Neighborhood'},inplace=True)

In [38]:
# Print rows of dataframe
df_combined.shape

(103, 3)